In [2]:
import os
import csv
import MDAnalysis
import subprocess
import Bio.PDB
import pandas as pd
import numpy as np
import MDAnalysis.analysis.hbonds

# Dssp (dssp.exe -i 1crn.pdb -o 1crn.dssp)
path = '/home/tongwade780/test_837_cluster'
path_csv = '/home/tongwade780/test_837_cluster/dssp/csv'
#path = '/home/tongwade780/test_20_datas/test_internal_pdb/residTOinternal_pdb/'
#path_csv = '/home/tongwade780/test_20_datas/test_internal_pdb/residTOinternal_pdb/dssp/csv'
dssp_exec ='/opt/dssp/dssp'
files = os.listdir(path)
for onefile in files:
    if onefile[-3:] == 'pdb':
        #print onefile
        idx = pd.IndexSlice
        
        # Use DSSP to assign secondary structure type to each residue.
        pdb_parser = Bio.PDB.PDBParser()
        structure = pdb_parser.get_structure('PDB', os.path.join(path,onefile))
        model = structure[0]
        dssp_results = Bio.PDB.DSSP(model, os.path.join(path,onefile), dssp=dssp_exec)
        
        Uni = MDAnalysis.Universe(os.path.join(path,onefile))
        protein = Uni.select_atoms("protein")
        protein_chain = np.unique(protein.segids)[0]
        
        chain_ID_resid_list = dssp_results.keys()
        dssp_results_table = pd.DataFrame(list(dssp_results))
        dssp_results_table = dssp_results_table[[0,1,2,3,4,5]]
        dssp_results_table.columns = ['DSSP Index', 'Residue Type', 'SS', 'Acc', 'Phi', 'Psi']
        dssp_results_table.index = pd.MultiIndex.from_tuples([(chain_ID_resid_list[x][0], chain_ID_resid_list[x][1][1]) for x in range(len(chain_ID_resid_list))])
        dssp_results_table = dssp_results_table.reindex()
        
        # Make sure all helices have negative phi and psi angles, otherwise change them to '-'
        helix_only_rows = (dssp_results_table['SS'] == 'H')
        helix_only_table = dssp_results_table[helix_only_rows]
        first_chain_helix_only = helix_only_table.loc[protein_chain,:]
        pos_phi_or_psi_idx = first_chain_helix_only[(first_chain_helix_only['Phi'] >= 0).values + (first_chain_helix_only['Psi'] >= 0).values].index.tolist()
        dssp_results_table.loc[idx[protein_chain,pos_phi_or_psi_idx], 'SS'] = '-'
        
        with open(os.path.join(path_csv,onefile[:-4]+'_dssp.csv'), 'wb') as output_file:
            output_file_csv = csv.writer(output_file)
            for i in range(len(dssp_results_table['SS'])):
                if dssp_results_table['SS'][i] == 'H' or dssp_results_table['SS'][i] == 'I' or dssp_results_table['SS'][i] == 'G':
                    output_file_csv.writerow([dssp_results_table["DSSP Index"][i],0.5])
                elif dssp_results_table['SS'][i] == 'T' or dssp_results_table['SS'][i] == 'S':
                    output_file_csv.writerow([dssp_results_table["DSSP Index"][i],1])
                elif dssp_results_table['SS'][i] == 'B' or dssp_results_table['SS'][i] == 'E':
                    output_file_csv.writerow([dssp_results_table["DSSP Index"][i],0])
                else:
                    output_file_csv.writerow([dssp_results_table["DSSP Index"][i],0])
                